In [1]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

In [2]:
df = pd.read_csv("NewsDataset.csv")
df.head()

,Sr. No,Newspaper Name,Published Date,URL,Headline,Content,Human Summary,Category
0,1,The Hindu,2023-12-01,https://www.thehindu.com/news/national/sample-...,"""India Launches Chandrayaan-4 Successfully""","India successfully launched Chandrayaan-4, aim...",India launched Chandrayaan-4 to study the moon...,Science and Technology
1,2,Hindustan Times,2022-08-15,https://www.hindustantimes.com/india/sample-ne...,"""PM Announces Digital India 2.0 on Independenc...",The Prime Minister unveiled the Digital India ...,"PM launched Digital India 2.0, focusing on tec...",National News
2,3,Indian Express,2021-04-10,https://www.indianexpress.com/news/sample-news-3,"""Economic Growth Rebounds in Q1 2021""",India’s GDP showed a rebound in the first quar...,"India's Q1 2021 GDP rebounded, indicating a re...",Business and Finance
3,4,The Telegraph,2023-05-18,https://www.telegraphindia.com/nation/sample-n...,"""Cyclone Yaas Causes Widespread Damage in East...",Cyclone Yaas wreaked havoc in Odisha and West ...,Cyclone Yaas caused severe damage in Eastern I...,Environment
4,5,Deccan Chronicle,2020-10-05,https://www.deccanchronicle.com/nation/sample-...,"""Hyderabad Emerges as India’s Vaccine Hub""",Hyderabad became a central hub for COVID-19 va...,Hyderabad gained recognition as the COVID-19 v...,Health and Wellness


In [3]:
df.shape

(112, 8)

In [4]:
"""
here when i tried to generate sentence similarity score many article have 0 value bucause there is only 1 to 5 sentances.
so in this logic i tired to count the lenth of each article like how many sentances each article have. and latter remove 
the articles who has 
l = []
for i in range(0, len(df)):
    article = df['Content'][i]
    article = article.split(".")
    print(f"article: {article}, len: {len(article)}")
    l.append(len(article))
print(l)
"""

'\nhere when i tried to generate sentence similarity score many article have 0 value bucause there is only 1 to 5 sentances.\nso in this logic i tired to count the lenth of each article like how many sentances each article have. and latter remove \nthe articles who has \nl = []\nfor i in range(0, len(df)):\n    article = df[\'Content\'][i]\n    article = article.split(".")\n    print(f"article: {article}, len: {len(article)}")\n    l.append(len(article))\nprint(l)\n'

In [5]:
df1 = df[['Content','Human Summary']] # getting only data that i needed 
df1.head()

,Content,Human Summary
0,"India successfully launched Chandrayaan-4, aim...",India launched Chandrayaan-4 to study the moon...
1,The Prime Minister unveiled the Digital India ...,"PM launched Digital India 2.0, focusing on tec..."
2,India’s GDP showed a rebound in the first quar...,"India's Q1 2021 GDP rebounded, indicating a re..."
3,Cyclone Yaas wreaked havoc in Odisha and West ...,Cyclone Yaas caused severe damage in Eastern I...
4,Hyderabad became a central hub for COVID-19 va...,Hyderabad gained recognition as the COVID-19 v...


In [6]:
# spliting each articles into sentances 
def tokens(article):
    article = article.split(".") # getting sentences 
    sentences = [] # list to store final sentences 
    # loop to clean the sentences
    for sentence in article:
        """
        this will replace any string that is not a letter (numbers, string punchuations etc.) with space 
        then split the results with space so only words get appended in the list.
        """
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" ")) 
    return sentences
df1['tokens'] = df['Content'].apply(tokens)
df1.head()

C:\Users\cappr\AppData\Local\Temp\ipykernel_11136\3939767515.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['tokens'] = df['Content'].apply(tokens)


,Content,Human Summary,tokens
0,"India successfully launched Chandrayaan-4, aim...",India launched Chandrayaan-4 to study the moon...,"[[India, successfully, launched, Chandrayaan-4..."
1,The Prime Minister unveiled the Digital India ...,"PM launched Digital India 2.0, focusing on tec...","[[The, Prime, Minister, unveiled, the, Digital..."
2,India’s GDP showed a rebound in the first quar...,"India's Q1 2021 GDP rebounded, indicating a re...","[[India’s, GDP, showed, a, rebound, in, the, f..."
3,Cyclone Yaas wreaked havoc in Odisha and West ...,Cyclone Yaas caused severe damage in Eastern I...,"[[Cyclone, Yaas, wreaked, havoc, in, Odisha, a..."
4,Hyderabad became a central hub for COVID-19 va...,Hyderabad gained recognition as the COVID-19 v...,"[[Hyderabad, became, a, central, hub, for, COV..."


In [7]:
"""
here when i tried to generate sentence similarity score of many article have 0 value bucause there is only 1 to 5 sentances
and every sentence is different.so in this logic i tired to count the lenth of each article tokens (my sentences) 
like how many sentances each article have. and latter remove the articles who has less than 5 sentances to get better simlilarity scores.
"""
def lenth(content):
    return len(content)
df1['len'] = df1['tokens'].apply(lenth)
df1.head()

C:\Users\cappr\AppData\Local\Temp\ipykernel_11136\20342150.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['len'] = df1['tokens'].apply(lenth)


,Content,Human Summary,tokens,len
0,"India successfully launched Chandrayaan-4, aim...",India launched Chandrayaan-4 to study the moon...,"[[India, successfully, launched, Chandrayaan-4...",2
1,The Prime Minister unveiled the Digital India ...,"PM launched Digital India 2.0, focusing on tec...","[[The, Prime, Minister, unveiled, the, Digital...",3
2,India’s GDP showed a rebound in the first quar...,"India's Q1 2021 GDP rebounded, indicating a re...","[[India’s, GDP, showed, a, rebound, in, the, f...",2
3,Cyclone Yaas wreaked havoc in Odisha and West ...,Cyclone Yaas caused severe damage in Eastern I...,"[[Cyclone, Yaas, wreaked, havoc, in, Odisha, a...",2
4,Hyderabad became a central hub for COVID-19 va...,Hyderabad gained recognition as the COVID-19 v...,"[[Hyderabad, became, a, central, hub, for, COV...",2


In [8]:
# removing articles who has less then 5 sentences
df1 = df1[df1['len'] >= 5]
df1 = df1.reset_index(drop=True)
df1.drop(columns='len')
df1.head()

,Content,Human Summary,tokens,len
0,Invitees to the “Pran Pratishtha” consecration...,The consecration ceremony of the Ram temple in...,"[[Invitees, to, the, “Pran, Pratishtha”, conse...",8
1,The sixth edition of The Hindu businessline Ch...,The sixth edition of The Hindu Businessline Ch...,"[[The, sixth, edition, of, The, Hindu, busines...",27
2,"ITANAGAR, Jan 02: In a tragic incident, at lea...",Six army jawans from the 11th Garhwal Regiment...,"[[ITANAGAR,, Jan, 02:, In, a, tragic, incident...",5
3,The Gujarat Police files First Information Rep...,The Gujarat Police filed an FIR under IPC Sect...,"[[The, Gujarat, Police, files, First, Informat...",8
4,Bonus shares in October 2022: After the arriva...,"In October 2022, five companies will issue bon...","[[Bonus, shares, in, October, 2022:, After, th...",31


In [9]:
# not a big diffrent only 12 articles were dropped 
df1.shape

(100, 4)

In [10]:
# computing the sentence similarity
def sentence_similarity(sent1, sent2 ):
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    words = list(set(sent1 + sent2))
    vector1 = [0] * len(words)
    vector2 = [0] * len(words)
    # build the vector for the first sentence
    for w in sent1:
        vector1[words.index(w)] += 1
    # build the vector for the second sentence
    for w in sent2:
        vector2[words.index(w)] += 1
    return 1 - cosine_distance(vector1, vector2)

In [11]:
def sim_max(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2])
    return similarity_matrix
df1['sim_max'] = df1['tokens'].apply(sim_max)

In [12]:
df1.head(50)

,Content,Human Summary,tokens,len,sim_max
0,Invitees to the “Pran Pratishtha” consecration...,The consecration ceremony of the Ram temple in...,"[[Invitees, to, the, “Pran, Pratishtha”, conse...",8,"[[0.0, 0.524872353392131, 0.3474456257081473, ..."
1,The sixth edition of The Hindu businessline Ch...,The sixth edition of The Hindu Businessline Ch...,"[[The, sixth, edition, of, The, Hindu, busines...",27,"[[0.0, 0.23267002029884432, 0.2608969660436018..."
2,"ITANAGAR, Jan 02: In a tragic incident, at lea...",Six army jawans from the 11th Garhwal Regiment...,"[[ITANAGAR,, Jan, 02:, In, a, tragic, incident...",5,"[[0.0, 0.10079052613579398, 0.2721655269759087..."
3,The Gujarat Police files First Information Rep...,The Gujarat Police filed an FIR under IPC Sect...,"[[The, Gujarat, Police, files, First, Informat...",8,"[[0.0, 0.17739371879672472, 0.3849741916091624..."
4,Bonus shares in October 2022: After the arriva...,"In October 2022, five companies will issue bon...","[[Bonus, shares, in, October, 2022:, After, th...",31,"[[0.0, 0.2954684201426394, 0.17496355305594125..."
5,The CBI summoned five doctors and some junior ...,The CBI summoned five doctors and junior docto...,"[[The, CBI, summoned, five, doctors, and, some...",12,"[[0.0, 0.19069251784911845, 0.3775519229794155..."
6,Border posts of the Indo-Tibetan Border Police...,\nThe Indian government has approved a Border ...,"[[Border, posts, of, the, Indo-Tibetan, Border...",23,"[[0.0, 0.3142857142857143, 0.3450327796711772,..."
7,The special POCSO court here in East Siang dis...,"Kalen Apang, 33, from East Siang district, Ind...","[[The, special, POCSO, court, here, in, East, ...",12,"[[0.0, 0.26160832178497173, 0.2754113163776217..."
8,"GUWAHATI, 27 Aug: Restless victims of Manipur’...","Victims of ethnic strife in Manipur, housed in...","[[GUWAHATI,, 27, Aug:, Restless, victims, of, ...",37,"[[0.0, 0.21759706994462236, 0.2823298512866399..."
9,"ITANAGAR, 19 Jan: Arunachal Pradesh Congress p...",Arunachal Pradesh Congress president Nabam Tuk...,"[[ITANAGAR,, 19, Jan:, Arunachal, Pradesh, Con...",15,"[[0.0, 0.1283881477532739, 0.21957751641341994..."


In [13]:
df1['tokens'][0]

[['Invitees',
  'to',
  'the',
  '“Pran',
  'Pratishtha”',
  'consecration',
  'ceremony',
  'at',
  'the',
  'Ram',
  'temple',
  'started',
  'arriving',
  'on',
  'January',
  '21',
  'in',
  'Ayodhya',
  '—',
  'once',
  'a',
  'sleepy',
  'town',
  'now',
  'resplendent',
  'with',
  'new',
  'infrastructure',
  'and',
  'gripped',
  'by',
  'heightened',
  'religious',
  'fervour',
  'on',
  'the',
  'eve',
  'of',
  'a',
  'seminal',
  'event',
  'in',
  'India’s',
  'political',
  'and',
  'religious',
  'history'],
 ['',
  'A',
  'multi-layer',
  'security',
  'cover',
  'is',
  'in',
  'place',
  'in',
  'Ayodhya',
  'for',
  'the',
  'consecration',
  'ceremony',
  'at',
  'the',
  'Ram',
  'temple,',
  'with',
  '10,000',
  'CCTVs',
  'keeping',
  'an',
  'eye',
  'on',
  'the',
  'movement',
  'of',
  'people',
  'and',
  'police',
  'personnel',
  'in',
  'plain',
  'clothes',
  'deployed',
  'at',
  'the',
  'venue'],
 ['',
  'From',
  'Dharampath',
  'and',
  'Rampath',

In [14]:
# Ranking sentences in similarity martix
def scores(s_m):
    matrix = s_m
    sentence_similarity_graph = nx.from_numpy_array(matrix)
    score = nx.pagerank(sentence_similarity_graph)
    return score
df1['score'] = df1['sim_max'].apply(scores)
df1.head()

,Content,Human Summary,tokens,len,sim_max,score
0,Invitees to the “Pran Pratishtha” consecration...,The consecration ceremony of the Ram temple in...,"[[Invitees, to, the, “Pran, Pratishtha”, conse...",8,"[[0.0, 0.524872353392131, 0.3474456257081473, ...","{0: 0.14309997824360216, 1: 0.1526298064795244..."
1,The sixth edition of The Hindu businessline Ch...,The sixth edition of The Hindu Businessline Ch...,"[[The, sixth, edition, of, The, Hindu, busines...",27,"[[0.0, 0.23267002029884432, 0.2608969660436018...","{0: 0.041586406717606036, 1: 0.042121643093350..."
2,"ITANAGAR, Jan 02: In a tragic incident, at lea...",Six army jawans from the 11th Garhwal Regiment...,"[[ITANAGAR,, Jan, 02:, In, a, tragic, incident...",5,"[[0.0, 0.10079052613579398, 0.2721655269759087...","{0: 0.20628506838306762, 1: 0.1606268382905125..."
3,The Gujarat Police files First Information Rep...,The Gujarat Police filed an FIR under IPC Sect...,"[[The, Gujarat, Police, files, First, Informat...",8,"[[0.0, 0.17739371879672472, 0.3849741916091624...","{0: 0.1267371772574894, 1: 0.10732110436335836..."
4,Bonus shares in October 2022: After the arriva...,"In October 2022, five companies will issue bon...","[[Bonus, shares, in, October, 2022:, After, th...",31,"[[0.0, 0.2954684201426394, 0.17496355305594125...","{0: 0.03353676253495321, 1: 0.0309747153613235..."


In [15]:
def rank(row):
    # Extract tokens and score dictionary for the current row
    tokens = row['tokens']  # List of tokens
    scores = row['score']   # Dictionary of token:score pairs
    
    # Pair each token with its score and sort by score in descending order
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(tokens)), reverse=True)
    return ranked_sentence


df1['ranked_sen'] = df1.apply(rank, axis=1)
df1.head()

,Content,Human Summary,tokens,len,sim_max,score,ranked_sen
0,Invitees to the “Pran Pratishtha” consecration...,The consecration ceremony of the Ram temple in...,"[[Invitees, to, the, “Pran, Pratishtha”, conse...",8,"[[0.0, 0.524872353392131, 0.3474456257081473, ...","{0: 0.14309997824360216, 1: 0.1526298064795244...","[(0.16952095489641944, [, The, clarification, ..."
1,The sixth edition of The Hindu businessline Ch...,The sixth edition of The Hindu Businessline Ch...,"[[The, sixth, edition, of, The, Hindu, busines...",27,"[[0.0, 0.23267002029884432, 0.2608969660436018...","{0: 0.041586406717606036, 1: 0.042121643093350...","[(0.05624370637812645, [, This, award, is, not..."
2,"ITANAGAR, Jan 02: In a tragic incident, at lea...",Six army jawans from the 11th Garhwal Regiment...,"[[ITANAGAR,, Jan, 02:, In, a, tragic, incident...",5,"[[0.0, 0.10079052613579398, 0.2721655269759087...","{0: 0.20628506838306762, 1: 0.1606268382905125...","[(0.25216098245959095, [, , One, of, the, conv..."
3,The Gujarat Police files First Information Rep...,The Gujarat Police filed an FIR under IPC Sect...,"[[The, Gujarat, Police, files, First, Informat...",8,"[[0.0, 0.17739371879672472, 0.3849741916091624...","{0: 0.1267371772574894, 1: 0.10732110436335836...","[(0.17999716023502005, [, Nine, persons, of, t..."
4,Bonus shares in October 2022: After the arriva...,"In October 2022, five companies will issue bon...","[[Bonus, shares, in, October, 2022:, After, th...",31,"[[0.0, 0.2954684201426394, 0.17496355305594125...","{0: 0.03353676253495321, 1: 0.0309747153613235...","[(0.049158467129749035, [, The, board, of, dir..."


In [16]:
df1['ranked_sen'][0]

[(0.16952095489641944,
  ['',
   'The',
   'clarification',
   'was',
   'issued',
   'a',
   'day',
   'after',
   'a',
   'circular',
   'said',
   'that',
   'AIIMS',
   'would',
   'observe',
   'a',
   'half-day',
   'closure',
   'of',
   'its',
   'offices',
   'and',
   'institutions',
   'on',
   'January',
   '22,',
   'in',
   'view',
   'of',
   'the',
   'consecration',
   'ceremony',
   'of',
   'the',
   'idol',
   'at',
   'Ayodhya’s',
   'Ram',
   'temple']),
 (0.1526298064795244,
  ['',
   'A',
   'multi-layer',
   'security',
   'cover',
   'is',
   'in',
   'place',
   'in',
   'Ayodhya',
   'for',
   'the',
   'consecration',
   'ceremony',
   'at',
   'the',
   'Ram',
   'temple,',
   'with',
   '10,000',
   'CCTVs',
   'keeping',
   'an',
   'eye',
   'on',
   'the',
   'movement',
   'of',
   'people',
   'and',
   'police',
   'personnel',
   'in',
   'plain',
   'clothes',
   'deployed',
   'at',
   'the',
   'venue']),
 (0.14309997824360216,
  ['Invitees',
  

In [17]:
# displaying top 3 sentences as summary
def get_sum(ranked_sen):
    ranked_sentence = ranked_sen
    n = 3
    summarize_text = []
    for i in range(n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    return ". ".join(summarize_text)

df1['Summary'] = df1['ranked_sen'].apply(get_sum)
df1.head()

,Content,Human Summary,tokens,len,sim_max,score,ranked_sen,Summary
0,Invitees to the “Pran Pratishtha” consecration...,The consecration ceremony of the Ram temple in...,"[[Invitees, to, the, “Pran, Pratishtha”, conse...",8,"[[0.0, 0.524872353392131, 0.3474456257081473, ...","{0: 0.14309997824360216, 1: 0.1526298064795244...","[(0.16952095489641944, [, The, clarification, ...",The clarification was issued a day after a ci...
1,The sixth edition of The Hindu businessline Ch...,The sixth edition of The Hindu Businessline Ch...,"[[The, sixth, edition, of, The, Hindu, busines...",27,"[[0.0, 0.23267002029884432, 0.2608969660436018...","{0: 0.041586406717606036, 1: 0.042121643093350...","[(0.05624370637812645, [, This, award, is, not...",This award is not part of the jury process th...
2,"ITANAGAR, Jan 02: In a tragic incident, at lea...",Six army jawans from the 11th Garhwal Regiment...,"[[ITANAGAR,, Jan, 02:, In, a, tragic, incident...",5,"[[0.0, 0.10079052613579398, 0.2721655269759087...","{0: 0.20628506838306762, 1: 0.1606268382905125...","[(0.25216098245959095, [, , One, of, the, conv...",One of the convoy truck carrying eight jawan...
3,The Gujarat Police files First Information Rep...,The Gujarat Police filed an FIR under IPC Sect...,"[[The, Gujarat, Police, files, First, Informat...",8,"[[0.0, 0.17739371879672472, 0.3849741916091624...","{0: 0.1267371772574894, 1: 0.10732110436335836...","[(0.17999716023502005, [, Nine, persons, of, t...","Nine persons of the Oreva group, involved in ..."
4,Bonus shares in October 2022: After the arriva...,"In October 2022, five companies will issue bon...","[[Bonus, shares, in, October, 2022:, After, th...",31,"[[0.0, 0.2954684201426394, 0.17496355305594125...","{0: 0.03353676253495321, 1: 0.0309747153613235...","[(0.049158467129749035, [, The, board, of, dir...",The board of directors of the company has fix...


In [18]:
# display options to show full content
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.width', None)  # Prevent line wrapping

df1[['Human Summary', 'Summary']].head(10)

,Human Summary,Summary
0,"The consecration ceremony of the Ram temple in Ayodhya is taking place on January 22. Thousands of devotees are arriving in the town, which has been transformed with new infrastructure. A massive security operation is in place to ensure the safety of the event. The AIIMS hospital in Delhi initially announced a half-day closure for January 22 but later clarified that critical clinical services would remain open due to public backlash.","The clarification was issued a day after a circular said that AIIMS would observe a half-day closure of its offices and institutions on January 22, in view of the consecration ceremony of the idol at Ayodhya’s Ram temple. A multi-layer security cover is in place in Ayodhya for the consecration ceremony at the Ram temple, with 10,000 CCTVs keeping an eye on the movement of people and police personnel in plain clothes deployed at the venue. Invitees to the “Pran Pratishtha” consecration ceremony at the Ram temple started arriving on January 21 in Ayodhya — once a sleepy town now resplendent with new infrastructure and gripped by heightened religious fervour on the eve of a seminal event in India’s political and religious history"
1,"The sixth edition of The Hindu Businessline Changemaker Awards will be held in New Delhi on September 13, 2024, with Finance Minister Nirmala Sitharaman presenting awards across six categories, including a new Chairperson's Award for education. Over 950 nominations were evaluated by Deloitte, Ashoka, and Niiti Consulting, with an independent jury chaired by R. Seshasayee selecting the winners. Supported by SASTRA University and SBI, the event will feature a musical performance and live-streaming at 5:30 p.m. on thbl.news/BCLM2024LS. The ceremony takes place at ITC Maurya, New Delhi.","This award is not part of the jury process that other awards have gone through and is given to an achiever in the field of education. At last year’s ceremony, the Reserve Bank of India received the Changemaker of the Year award while GCMMF (Amul) was named Iconic Changemaker of the Year. Casagrand is the realty partner, while Fortinet is the cyber security partner"
2,"Six army jawans from the 11th Garhwal Regiment were killed, and two were injured when their convoy truck veered off the road and plunged into a gorge near Sessa in West Kameng. The truck, hit by a falling rock, was en route from Sessa to Bhalukpong. Injured jawans were hospitalized.","One of the convoy truck carrying eight jawans was hit by shooting stone, plunged into deep gorge after veered off the road, source added. ITANAGAR, Jan 02: In a tragic incident, at least six army jawans of 11th Garhwal Regiment of Gajraj Corps were killed and two others injured when their convoy truck was reportedly plunged into deep gorge near Sessa in West Kameng yesterday. Ram Prasad and injured jawans were identified as Kuldeep Singh and Kundun, both were rushed to Army Hospital at Tezpur (Assam)"
3,"The Gujarat Police filed an FIR under IPC Sections 304 and 308 for culpable homicide in the Morbi bridge collapse, which claimed 141 lives and injured over 100. Nine members of the Oreva group, responsible for the bridge's maintenance, were arrested. Rescue operations are ongoing, with NDRF, Navy, and Air Force teams deployed. PM Modi will visit the site on November 1 to oversee relief efforts and ensure no laxity in rescue operations.","Nine persons of the Oreva group, involved in the maintenance and management of the bridge, were arrested on October 31 in connection with the collapse of the cable suspension bridge on the Machchhu river. Police have registered an FIR on charges of culpable homicide against agencies given the task of maintenance and operation of the bridge in Morbi city which collapsed on Sunday evening. Several teams of NDRF and personnel of Navy and Air Force have been ordered to be airlifted to the site to help in the rescue efforts and comb the river bed as many people are feared drowned in the 10 